In [1]:
import pandas as pd

In [2]:
stu_path = pd.read_csv('Resources/students_complete.csv', delimiter = ',')
sch_path = pd.read_csv('Resources/schools_complete.csv', delimiter = ',')

In [3]:
sch_path.columns

Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object')

In [4]:
stu_path.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score'],
      dtype='object')

In [5]:
#Sum of the total budget of all schools
total_budget = sch_path["budget"].sum()
total_budget

24649428

In [6]:
#Count of total number of schools
total_schools = sch_path["school_name"].count()
total_schools

15

In [7]:
#Count Total Students
total_students = stu_path["student_name"].count()
total_students

39170

In [8]:
#Average Math Scores
avg_math_scores = stu_path["math_score"].mean()
avg_math_scores = "{:.2f}".format(avg_math_scores)
avg_math_scores

'78.99'

In [9]:
#Average Reading Scores
avg_reading_scores = stu_path["reading_score"].mean()
avg_reading_scores = "{:.2f}".format(avg_reading_scores)
avg_reading_scores

'81.88'

In [10]:
#Total Students Passing Math
students_pass_math = stu_path.loc[stu_path["math_score"] > 65, "student_name"]
students_pass_math.count()

32555

In [11]:
#Total Students Passing Reading
students_pass_reading = stu_path.loc[stu_path["reading_score"] > 65, "student_name"]
students_pass_reading.count()

36923

In [12]:
#Percentage of Students Passing Math
percent_pass_math = (len(students_pass_math)/total_students)*100
percent_pass_math_string = "{:.2f}".format(percent_pass_math)
percent_pass_math_aspercent = f'{percent_pass_math_string} %'
percent_pass_math_aspercent

'83.11 %'

In [13]:
#Percentage of Students Passing Reading
percent_pass_reading = (len(students_pass_reading)/total_students)*100
percent_pass_reading_string = "{:.2f}".format(percent_pass_reading)
percent_pass_reading_aspercent = f'{percent_pass_reading_string} %'
percent_pass_reading_aspercent 
# = "{:.2f}".format(percent_pass_reading)

'94.26 %'

In [14]:
#Average of both Math & Reading Averages
Average_of_scores = ((percent_pass_math + percent_pass_reading)/2)
Average_of_scores_string = "{:.2f}".format(Average_of_scores)
Average_of_scores_aspercent = f'{Average_of_scores_string} %'
Average_of_scores_aspercent

'88.69 %'

In [15]:
#creating Data Frame Table for district schools
district_table = [{"Total Budget":total_budget, "Total Schools":total_schools, "Total Students":total_students, 
                  "Average Math Scores":avg_math_scores, "Average Reading Scores":avg_reading_scores, "Passing Students (Math)":students_pass_math.count(),
                  "Passing Students (Reading)":students_pass_reading.count(),"Percent Pass (Math)":percent_pass_math_aspercent,
                  "Percent Pass (Reading)":percent_pass_reading_aspercent, "Average of Reading & Math Scores":Average_of_scores_aspercent}]
dist_table_df = pd.DataFrame(district_table)
dist_table_df

,Total Budget,Total Schools,Total Students,Average Math Scores,Average Reading Scores,Passing Students (Math),Passing Students (Reading),Percent Pass (Math),Percent Pass (Reading),Average of Reading & Math Scores
0,24649428,15,39170,78.99,81.88,32555,36923,83.11 %,94.26 %,88.69 %


In [16]:
#COMBINED DF CELL

#Merging two df's on school_name

combined_df = pd.merge(sch_path, stu_path, on = "school_name", how = "outer")

combined_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [17]:
#Re-naming Columns in DataFrame


combined_df.rename(columns={"school_name" : "School Name","type" : "Type","student_name":"Student Name", 
                            "reading_score":"Reading Score","math_score":"Math Score", "budget":"Budget","size":"Size"}, inplace = True)
pass_math_students_by_sch = combined_df.loc[combined_df["Math Score"] > 70, ["School Name"]]
pass_read_students_by_sch = combined_df.loc[combined_df["Reading Score"] > 70, ["School Name"]]

combined_df["Passing Students Math"] = pass_math_students_by_sch
combined_df["Passing Students Reading"] = pass_read_students_by_sch

In [18]:
#Using Value_counts to find how many students have passed per school.

pass_math_students_by_sch = combined_df["Passing Students Math"].value_counts()
pass_read_students_by_sch = combined_df["Passing Students Reading"].value_counts()

In [19]:
#Building Data Frame to Give Data per school

#Groupby School Name. That will be our left-most column and our way of sorting data
combined_df_groupby = combined_df.groupby(["School Name"])
average_scores_schools = combined_df_groupby["Budget","Size","Reading Score", "Math Score"].mean()
#Renaming the Headers to indicate Average's were calculated
average_scores_schools.rename(columns={"Reading Score":"Reading Score (Average)", "Math Score":"Math Score (Average)"},inplace = True)

#Calculating budget per student for each school
budget_per_student = average_scores_schools ["Budget"] / average_scores_schools ["Size"]
budget_per_student_df = pd.DataFrame({"Budget":budget_per_student})


#Appending a new column/series to the data frame
average_scores_schools["Budget Per Student"] = budget_per_student_df

#Calculating Passing Rate per Subject
passrate_math_per_sch = (pass_math_students_by_sch / average_scores_schools ["Size"])*100
passrate_math_per_sch

passrate_reading_per_sch= (pass_read_students_by_sch / average_scores_schools ["Size"])*100
passrate_reading_per_sch


#Calculating Overall Passing Rate Per School
overall_pass_rate_per_sch = (passrate_math_per_sch + passrate_reading_per_sch)/2

# #Appending a new column/series to the data frame
average_scores_schools["Passing Rate (Math)"] = passrate_math_per_sch
average_scores_schools["Passing Rate (Reading)"] = passrate_reading_per_sch
average_scores_schools["Passing Rate (Overall)"] = overall_pass_rate_per_sch

average_scores_schools

,Budget,Size,Reading Score (Average),Math Score (Average),Budget Per Student,Passing Rate (Math),Passing Rate (Reading),Passing Rate (Overall)
School Name,,,,,,,,
Bailey High School,3124928.0,4976.0,81.033963,77.048432,628.0,64.630225,79.300643,71.965434
Cabrera High School,1081356.0,1858.0,83.975780,83.061895,582.0,89.558665,93.864370,91.711518
Figueroa High School,1884411.0,2949.0,81.158020,76.711767,639.0,63.750424,78.433367,71.091896
Ford High School,1763916.0,2739.0,80.746258,77.102592,644.0,65.753925,77.510040,71.631982
Griffin High School,917500.0,1468.0,83.816757,83.351499,625.0,89.713896,93.392371,91.553134
Hernandez High School,3022020.0,4635.0,80.934412,77.289752,652.0,64.746494,78.187702,71.467098
Holden High School,248087.0,427.0,83.814988,83.803279,581.0,90.632319,92.740047,91.686183
Huang High School,1910635.0,2917.0,81.182722,76.629414,655.0,63.318478,78.813850,71.066164
Johnson High School,3094650.0,4761.0,80.966394,77.072464,650.0,63.852132,78.281874,71.067003


In [20]:
#Top 5 schools based on Overall Passing Rate 
average_scores_schools [["Passing Rate (Overall)"]]
average_scores_schools.sort_values(["Passing Rate (Overall)"], ascending = False).head()

,Budget,Size,Reading Score (Average),Math Score (Average),Budget Per Student,Passing Rate (Math),Passing Rate (Reading),Passing Rate (Overall)
School Name,,,,,,,,
Wilson High School,1319574.0,2283.0,83.989488,83.274201,578.0,90.932983,93.254490,92.093736
Pena High School,585858.0,962.0,84.044699,83.839917,609.0,91.683992,92.203742,91.943867
Wright High School,1049400.0,1800.0,83.955000,83.682222,583.0,90.277778,93.444444,91.861111
Cabrera High School,1081356.0,1858.0,83.975780,83.061895,582.0,89.558665,93.864370,91.711518
Holden High School,248087.0,427.0,83.814988,83.803279,581.0,90.632319,92.740047,91.686183


In [21]:
#Bottom 5 schools based on Overall Passing Rate 

average_scores_schools.sort_values(["Passing Rate (Overall)"], ascending = True).head()

,Budget,Size,Reading Score (Average),Math Score (Average),Budget Per Student,Passing Rate (Math),Passing Rate (Reading),Passing Rate (Overall)
School Name,,,,,,,,
Rodriguez High School,2547363.0,3999.0,80.744686,76.842711,637.0,64.066017,77.744436,70.905226
Huang High School,1910635.0,2917.0,81.182722,76.629414,655.0,63.318478,78.813850,71.066164
Johnson High School,3094650.0,4761.0,80.966394,77.072464,650.0,63.852132,78.281874,71.067003
Figueroa High School,1884411.0,2949.0,81.158020,76.711767,639.0,63.750424,78.433367,71.091896
Hernandez High School,3022020.0,4635.0,80.934412,77.289752,652.0,64.746494,78.187702,71.467098


In [22]:
#Creating a new Groupby (2 series GroupBy) Function sorted by year group
                #READING SCORES ONLY DATAFRAME


combined_df_groupby_year_read = combined_df.drop(["Size","Budget","Type","Student ID","gender", "School ID","Math Score"],axis=1)
combined_df_groupby_year_read = combined_df_groupby_year_read.groupby(["School Name","grade"])
combined_df_groupby_year_read.mean()


Reading Score
School Name           grade               
Bailey High School    10th       80.907183
                      11th       80.945643
                      12th       80.912451
                      9th        81.303155
Cabrera High School   10th       84.253219
                      11th       83.788382
                      12th       84.287958
                      9th        83.676136
Figueroa High School  10th       81.408912
                      11th       80.640339
                      12th       81.384863
                      9th        81.198598
Ford High School      10th       81.262712
                      11th       80.403642
                      12th       80.662338
                      9th        80.632653
Griffin High School   10th       83.706897
                      11th       84.288089
                      12th       84.013699
                      9th        83.369193
Hernandez High School 10th       80.660147
                      11th       81.396140
                      12th       80.857143
                      9th        80.866860
Holden High School    10th       83.324561
                      11th       83.815534
                      12th       84.698795
                      9th        83.677165
Huang High School     10th       81.512386
                      11th       81.417476
                      12th       80.305983
                      9th        81.290284
Johnson High School   10th       80.773431
                      11th       80.616027
                      12th       81.227564
                      9th        81.260714
Pena High School      10th       83.612000
                      11th       84.335938
                      12th       84.591160
                      9th        83.807273
Rodriguez High School 10th       80.629808
                      11th       80.864811
                      12th       80.376426
                      9th        80.993127
Shelton High School   10th       83.441964
                      11th       84.373786
                      12th       82.781671
                      9th        84.122642
Thomas High School    10th       84.254157
                      11th       83.585542
                      12th       83.831361
                      9th        83.728850
Wilson High School    10th       84.021452
                      11th       83.764608
                      12th       84.317673
                      9th        83.939778
Wright High School    10th       83.812757
                      11th       84.156322
                      12th       84.073171
                      9th        83.833333

In [23]:
#Creating a new Groupby (2 series GroupBy) Function sorted by year group
                #MATH SCORES ONLY DATAFRAME


combined_df_groupby_year_math = combined_df.drop(["Size","Budget","Type","Student ID","gender", "School ID","Reading Score"],axis=1)
combined_df_groupby_year_math = combined_df_groupby_year_math.groupby(["School Name","grade"])
combined_df_groupby_year_math.mean()

Math Score
School Name           grade            
Bailey High School    10th    76.996772
                      11th    77.515588
                      12th    76.492218
                      9th     77.083676
Cabrera High School   10th    83.154506
                      11th    82.765560
                      12th    83.277487
                      9th     83.094697
Figueroa High School  10th    76.539974
                      11th    76.884344
                      12th    77.151369
                      9th     76.403037
Ford High School      10th    77.672316
                      11th    76.918058
                      12th    76.179963
                      9th     77.361345
Griffin High School   10th    84.229064
                      11th    83.842105
                      12th    83.356164
                      9th     82.044010
Hernandez High School 10th    77.337408
                      11th    77.136029
                      12th    77.186567
                      9th     77.438495
Holden High School    10th    83.429825
                      11th    85.000000
                      12th    82.855422
                      9th     83.787402
Huang High School     10th    75.908735
                      11th    76.446602
                      12th    77.225641
                      9th     77.027251
Johnson High School   10th    76.691117
                      11th    77.491653
                      12th    76.863248
                      9th     77.187857
Pena High School      10th    83.372000
                      11th    84.328125
                      12th    84.121547
                      9th     83.625455
Rodriguez High School 10th    76.612500
                      11th    76.395626
                      12th    77.690748
                      9th     76.859966
Shelton High School   10th    82.917411
                      11th    83.383495
                      12th    83.778976
                      9th     83.420755
Thomas High School    10th    83.087886
                      11th    83.498795
                      12th    83.497041
                      9th     83.590022
Wilson High School    10th    83.724422
                      11th    83.195326
                      12th    83.035794
                      9th     83.085578
Wright High School    10th    84.010288
                      11th    83.836782
                      12th    83.644986
                      9th     83.264706

In [24]:
#DATAFRAME BASED ON SCHOOL SPENDING

#Creating a new series in Combined_df DataFrame
combined_df["Budget per Student"] = combined_df["Budget"] / combined_df ["Size"]

#Creating Bins for School Budget Per Student
bin_values = [0,599,630,645,660]
bin_label = ["Low","Below Average", "Above Average", "High"]
#Adding Newly Created Bin To Combined-DF
combined_df ["Funding Provided per Student"]= pd.cut(combined_df["Budget per Student"],bin_values, labels=bin_label)

#Creating Groupby Statement
combined_df_groupby_budget = combined_df.groupby(["Funding Provided per Student"])
#Finding Mean of Reading & Math Scores. Renamed the DataFrame
average_scores_by_budget = combined_df_groupby_budget["Reading Score", 'Math Score'].mean()
#Total Number of Students Based on School Spending
students_by_sch_budget = combined_df_groupby_budget["Funding Provided per Student"].count()

#Creating another Series for Students who passed based on School Spending 
combined_df ["Student Passing Reading by Budget (binned)"] = combined_df.loc[combined_df["Reading Score"] > 70, ["Funding Provided per Student"]]
#Finding Reading Passing Rate Per School Spending
passrate_reading_per_sch_budget= (combined_df["Student Passing Reading by Budget (binned)"].value_counts()/combined_df["Funding Provided per Student"].value_counts())*100
#Adding a new Series to the Renamed Data Frame
average_scores_by_budget ["Passing Rate Reading / Budget"] =passrate_reading_per_sch_budget


#Creating another Series for Students who passed based on School Spending
combined_df ["Student Passing Math by Budget (bins)"]= pd.cut(combined_df["Budget per Student"],bin_values, labels=bin_label)
students_by_sch_budget = combined_df_groupby_budget["Student Passing Math by Budget (bins)"].count()

combined_df ["Student Passing Math by Budget (binned)"] = combined_df.loc[combined_df["Math Score"] > 70, ["Student Passing Math by Budget (bins)"]]
#Finding Math Passing Rate Per School Spending
passrate_math_per_sch_budget= (combined_df["Student Passing Math by Budget (binned)"].value_counts()/combined_df["Student Passing Math by Budget (bins)"].value_counts())*100
#Adding a new Series to the Renamed Data Fram
average_scores_by_budget ["Passing Rate Math / Budget"] =passrate_math_per_sch_budget

#Finding Overall Pass Rate
overall_pass_rate_per_budget = (passrate_math_per_sch_budget + passrate_reading_per_sch_budget)/2


average_scores_by_budget ["Passing Rate Math / Budget"] =passrate_math_per_sch_budget
average_scores_by_budget ["Overall Passing Rate"] = overall_pass_rate_per_budget

average_scores_by_budget


,Reading Score,Math Score,Passing Rate Reading / Budget,Passing Rate Math / Budget,Overall Passing Rate
Funding Provided per Student,,,,,
Low,83.964039,83.363065,93.451633,90.326633,91.889133
Below Average,82.312643,79.982873,85.469619,76.339042,80.904331
Above Average,81.301007,77.821056,80.056527,68.168168,74.112348
High,81.005604,77.049297,78.372452,64.062373,71.217412


In [25]:
# #Assembling Data Frame for Size Bins

bin_values_size = [0,1519,3794,4977]
bin_label_size = ["Small","Medium", "Large"]
combined_df ["Student Count by School Size"] = pd.cut(combined_df["Size"], bin_values_size, labels=bin_label_size)


#create math pass by school size type (bins), add it to combined_df 
combined_df ["Student Passing Math by School Size (Bins)"] = combined_df.loc[combined_df["Math Score"] > 70, ["Student Count by School Size"]]

#create reading pass by school size type (bins), add it to combined_df 
combined_df ["Student Passing Reading by School Size (Bins)"] = combined_df.loc[combined_df["Reading Score"] > 70, ["Student Count by School Size"]]

#Groupby Size Bin
combined_df_groupby_size = combined_df.groupby(["Student Count by School Size"])
average_scores_by_size = combined_df_groupby_size["Reading Score", 'Math Score'].mean()

#Defining "Student Count by School Size"
students_by_sch_size = combined_df_groupby_size["Student Count by School Size"].count()

#Calculating Pass Rate
passrate_math_per_sch_size = (combined_df["Student Passing Math by School Size (Bins)"].value_counts()/combined_df["Student Count by School Size"].value_counts())*100

passrate_reading_per_sch_size = (combined_df["Student Passing Reading by School Size (Bins)"].value_counts()/combined_df["Student Count by School Size"].value_counts())*100

overall_pass_rate_per_sch = (passrate_math_per_sch_size + passrate_reading_per_sch_size)/2

average_scores_by_size["Passing Rate (Reading)"] = passrate_reading_per_sch_size
average_scores_by_size["Passing Rate (Math)"] = passrate_math_per_sch_size
average_scores_by_size["Overall Passing Rate"] = overall_pass_rate_per_sch
average_scores_by_size

,Reading Score,Math Score,Passing Rate (Reading),Passing Rate (Math),Overall Passing Rate
Student Count by School Size,,,,,
Small,83.893245,83.583479,92.894645,90.514526,91.704585
Medium,82.529094,80.213577,86.055066,77.756103,81.905585
Large,80.928365,77.070764,78.417070,64.335093,71.376082


In [26]:
# DataFrame based on school type

#Grouping By School Type
combined_df_groupby_type = combined_df.groupby(["Type"])
average_scores_by_type = combined_df_groupby_type["Reading Score", "Math Score"].mean()
students_by_type = combined_df_groupby_type["Type"].count()

#Using Loc Function to Find Students who passed
pass_math_students_by_sch_type = combined_df.loc[combined_df["Math Score"] > 70, ["Type"]]
combined_df["Passing Students Math by School Type"] = pass_math_students_by_sch_type
pass_math_students_by_sch_type = combined_df["Passing Students Math by School Type"].value_counts()


pass_read_students_by_sch_type = combined_df.loc[combined_df["Reading Score"] > 70, ["Type"]]
combined_df["Passing Students Reading by School Type"] = pass_read_students_by_sch_type
pass_read_students_by_sch_type = combined_df["Passing Students Reading by School Type"].value_counts()

# Calculating Passing Rate per Subject
passrate_math_per_sch_type = (pass_math_students_by_sch_type/students_by_type)*100
passrate_reading_per_sch_type= (pass_read_students_by_sch_type / students_by_type)*100



#Calculating Overall Passing Rate Per School
overall_pass_rate_per_sch = (passrate_math_per_sch_type + passrate_reading_per_sch_type)/2

average_scores_by_type["Passing Rate (Math)"] = passrate_math_per_sch_type
average_scores_by_type["Passing Rate (Reading)"] = passrate_reading_per_sch_type
average_scores_by_type["Passing Rate (Overall)"] = overall_pass_rate_per_sch

#Final DF for Scores by School Type
average_scores_by_type

,Reading Score,Math Score,Passing Rate (Math),Passing Rate (Reading),Passing Rate (Overall)
Type,,,,,
Charter,83.902821,83.406183,90.282106,93.152370,91.717238
District,80.962485,76.987026,64.305308,78.369662,71.337485
